In [2]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import math
import matplotlib.pyplot as plt

def inverse_qft(circuit, n):
    """Applies the Inverse Quantum Fourier Transform to the first n qubits."""
    # We implement the exact inverse of the QFT circuit
    for qubit in range(n // 2):
        circuit.swap(qubit, n - qubit - 1)
    for j in range(n):
        for m in range(j):
            circuit.cp(-math.pi / float(2**(j-m)), m, j)
        circuit.h(j)

def c_amod15(power):
    """
    Returns a controlled gate that computes 7^power mod 15.
    This is hardcoded specifically for a=7, N=15.
    """
    U = QuantumCircuit(4)        
    for _ in range(power):
        # The operation 7*x mod 15 cycles the states:
        # 1 -> 7 -> 4 -> 13 -> 1
        # This specific permutation can be implemented with these swaps:
        U.swap(0, 1)
        U.swap(1, 2)
        U.swap(2, 3)
        for q in range(4):
            U.x(q)
    U = U.to_gate()
    U.name = f"7^{power} mod 15"
    return U.control()

# --- STEP 1: Setup Registers ---
# We need enough qubits for the counting register (precision) and the working register (N)
n_count = 8  # Register 1: 8 qubits for reading the period (M=256)
n_work = 4   # Register 2: 4 qubits to hold the number 15 (1111)

qc = QuantumCircuit(n_count + n_work, n_count)

# --- STEP 2: Initialize State ---
# Register 1: Initialize to superposition (Hadamard everything)
for q in range(n_count):
    qc.h(q)

# Register 2: Initialize to state |1> (0001) because a^0 = 1
qc.x(n_count) 

# --- STEP 3: Modular Exponentiation ---
# Apply controlled-U operations
# This corresponds to a^x mod N
for q in range(n_count):
    # The exponent for the control qubit q is 2^q
    # So we apply (7^(2^q)) mod 15
    qc.append(c_amod15(2**q), [q] + list(range(n_count, n_count + n_work)))

# --- STEP 4: Inverse QFT ---
# We apply the Inverse QFT to Register 1 to reveal the period frequency
inverse_qft(qc, n_count)

# --- STEP 5: Measurement ---
# Measure Register 1
qc.measure(range(n_count), range(n_count))

print("Circuit Diagram:")
# 'fold=-1' prevents the diagram from wrapping to the next line 
# so you can scroll horizontally to see the whole flow.
#print(qc.draw(output='text', fold=-1))
qc.draw(output='mpl')
plt.show()

# --- STEP 6: Run Simulation ---
print("Running simulation...")
simulator = AerSimulator()
compiled_circuit = transpile(qc, simulator)
job = simulator.run(compiled_circuit, shots=1000)
result = job.result()
counts = result.get_counts()

# --- STEP 7: Classical Analysis (Interpreting Results) ---
print("\nMeasurement Counts (Top results):")
sorted_counts = sorted(counts.items(), key=lambda item: item[1], reverse=True)
for output_bitstring, count in sorted_counts[:5]:
    # Convert bitstring to integer
    measured_int = int(output_bitstring, 2)
    print(f"Measured: {measured_int} (Binary: {output_bitstring}) - Count: {count}")
    
    # Analyze the Phase
    # We measured y. The phase is y / M (where M = 2^n_count = 256)
    phase = measured_int / (2**n_count)
    
    # Use generic explanation for what happened
    print(f"  -> Phase: {phase:.4f}")
    # Continued fractions usually go here to find r, 
    # but for this specific demo we can just peek at the denominators.
    from fractions import Fraction
    frac = Fraction(phase).limit_denominator(15)
    print(f"  -> Fraction Estimate: {frac.numerator}/{frac.denominator} (Likely r={frac.denominator})")
    print("-" * 30)

Circuit Diagram:
Running simulation...

Measurement Counts (Top results):
Measured: 0 (Binary: 00000000) - Count: 512
  -> Phase: 0.0000
  -> Fraction Estimate: 0/1 (Likely r=1)
------------------------------
Measured: 128 (Binary: 10000000) - Count: 488
  -> Phase: 0.5000
  -> Fraction Estimate: 1/2 (Likely r=2)
------------------------------
